Work on getting an IGC that models ecostress
============================================

In [1]:
from geocal import *
from ecostress import *
import h5py
import os

In [2]:
orb_fname = "/data/smyth/ecostress-test-data/latest/L1A_RAW_ATT_80005_20150124T204251_0100_01.h5.expected"
orb = HdfOrbit_Eci_TimeJ2000(orb_fname, "", "Ephemeris/time_j2000",
                             "Ephemeris/eci_position",
                             "Ephemeris/eci_velocity",
                             "Attitude/time_j2000",
                             "Attitude/quaternion")
rad_fname = "/data/smyth/ecostress-test-data/latest/ECOSTRESS_L1B_RAD_80005_001_20150124T204251_0100_01.h5.expected"
f = h5py.File(rad_fname, "r")
tmlist = f["/Time/line_start_time_j2000"][:]
tt = MeasuredTimeTable([Time.time_j2000(t) for t in tmlist])
datum = os.environ["AFIDS_VDEV_DATA"] + "/EGM96_20_x100.HLF"
srtm_dir = os.environ["ELEV_ROOT"]
dem = SrtmDem(srtm_dir,False, DatumGeoid96(datum))

In [3]:
# Initial camera, we will change this
cam_fname = "/data/smyth/ecostress-test-data/latest/l1_osp_dir/camera.xml"
cam_old = read_shelve(cam_fname)

In [4]:
def pixel_size(cam, ic):
    t, fc = tt.time(ic)
    od = orb.orbit_data(t)
    gp1 = od.surface_intersect(cam, fc, dem)
    fc.line = fc.line + 1
    gp2 = od.surface_intersect(cam, fc, dem)
    fc.line = fc.line - 1
    fc.sample = fc.sample + 1
    gp3 = od.surface_intersect(cam, fc, dem)
    print(distance(gp1, gp2))
    print(distance(gp1, gp3))

In [5]:
pixel_size(cam_old, ImageCoordinate(0,0))

70.38688759623848
76.768843193277


In [6]:
cam = read_shelve("camera.xml")

In [7]:
pixel_size(cam, ImageCoordinate(0,0))

39.153972143190735
40.18314730765374


In [8]:
print(cam.number_band)

6


In [9]:
32.2e-6 * 2.660 * rad_to_deg * 5400

26.500489777014444

In [10]:
173.88e-3 / 32.2e-6

5400.000000000001

In [11]:
2.66 * 1.181

3.1414600000000004

In [12]:
25.4 / 60 * 2 * pi

2.659881780039358

In [13]:
t0, fc = tt.time(ImageCoordinate(0,0))
od = orb.orbit_data(t0)

In [18]:
def od_scan(od, sa):
    '''Convert a orbit data w/o the scan mirror to one with the scan_angle (in radians)'''
    return QuaternionOrbitData(od.time, od.position_ci, od.velocity_ci, 
                               od.sc_to_ci * quat_rot_x(sa))
def scan_angle(time):
    # Not sure about this
    scan_min = -25.5 * deg_to_rad
    scan_max = 25.5 * deg_to_rad
    scan_time = 173.88e-3
    scan_rate = (scan_max - scan_min) / scan_time
    return (time - t0) * scan_rate + scan_min
    
def od_time(od, time):
    return od_scan(od, scan_angle(time))

In [26]:
gp1 = od_time(od, t0 + 5400 * 31.6e-6).surface_intersect(cam, fc, dem)
gp2 = od_time(od, t0 + 5401 * 31.6e-6).surface_intersect(cam, fc, dem)
print(distance(gp1,gp2))

80.7276609540291


In [16]:
print(cam.sc_look_vector(FrameCoordinate(256,0.5),4))

Look vector in spacecraft coordinates: 
  dir:    (-0.0120433, 0.00159458, 0.999926)
  length: 425.031 m



In [24]:
402000 / 5400

74.44444444444444